In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv("car_evaluation.csv")

header_list =  ['Buying', 'Maintenance', 'Doors', 'Persons', 'Luggage_Boot', 'Safety']

#Add Name To Column
df = pd.read_csv("car_evaluation.csv", names=header_list)
df

,Buying,Maintenance,Doors,Persons,Luggage_Boot,Safety
vhigh,vhigh,2,2,small,low,unacc
vhigh,vhigh,2,2,small,med,unacc
vhigh,vhigh,2,2,small,high,unacc
vhigh,vhigh,2,2,med,low,unacc
vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...
low,low,5more,more,med,med,good
low,low,5more,more,med,high,vgood
low,low,5more,more,big,low,unacc
low,low,5more,more,big,med,good


In [4]:
#Check for null values
df.isnull().any(axis = 1).sum()

0

In [5]:
for i in (df.columns):
    print(i)
    print(df[i].unique(), '\n')

Buying
['vhigh' 'high' 'med' 'low'] 

Maintenance
['2' '3' '4' '5more'] 

Doors
['2' '4' 'more'] 

Persons
['small' 'med' 'big'] 

Luggage_Boot
['low' 'med' 'high'] 

Safety
['unacc' 'acc' 'vgood' 'good'] 



In [6]:
def car_evaluation_Safety(x):
    if x=='unacc':
        return "unacceptable"
    elif x=='acc':
        return "acceptable"
    elif x=='vgood':
        return "Very_Good"
    else:
        return 'good'
    
def car_evaluation_Maintenance(x):
    if x=='2':
        return "low"
    elif x=='3':
        return "med"
    elif x=='4':
        return "high"
    elif x=='5more':
        return "very_high"
    

In [7]:
def fillmissing_object(table, col_name):
    table[col_name] = table[col_name].astype('object')
    table[col_name].fillna(table[col_name].value_counts().idxmax(), inplace=True)

#fill the categorical values
fillmissing_object(df, 'Buying')
fillmissing_object(df, 'Maintenance')
fillmissing_object(df, 'Doors')
fillmissing_object(df, 'Persons')
fillmissing_object(df, 'Luggage_Boot')
fillmissing_object(df, 'Safety')


In [8]:
df.Safety = df.Safety.apply(lambda x:car_evaluation_Safety(x))    
df.Maintenance = df['Maintenance'].apply(lambda x:car_evaluation_Maintenance(x))
df.isnull().any(axis = 1).sum()


0

In [9]:
for i in (df.columns):
    print(i)
    print(df[i].unique(), '\n')

Buying
['vhigh' 'high' 'med' 'low'] 

Maintenance
['low' 'med' 'high' 'very_high'] 

Doors
['2' '4' 'more'] 

Persons
['small' 'med' 'big'] 

Luggage_Boot
['low' 'med' 'high'] 

Safety
['unacceptable' 'acceptable' 'Very_Good' 'good'] 



In [10]:
from scipy import stats

def chi_sq_test(DF, var1, var2):
    dataset_table = pd.crosstab(DF[var1], DF[var2])
    print('contingency table between {} and {} is:\n'.format(var1, var2))
    print(dataset_table)
    val = stats.chi2_contingency(dataset_table)
    Observed_Values = dataset_table.values 
    Expected_Values = val[-1]
    no_of_rows = dataset_table.shape[0]
    no_of_columns = dataset_table.shape[1]
    ddof = (no_of_rows-1)*(no_of_columns-1)
    print("\nDegree of Freedom:-",ddof)
    alpha = 0.05
    chi_square = sum([(o-e)**2./e for o,e in zip(Observed_Values, Expected_Values)])
    chi_square_statistic = chi_square[0]+chi_square[1]
    critical_value = stats.chi2.ppf(q=1-alpha,df=ddof)
    p_value = 1-stats.chi2.cdf(x=chi_square_statistic,df=ddof)
    
    if chi_square_statistic>=critical_value and p_value<=alpha:
        print("chi_square_statistic is {} and p-value is {}.\nSo, there is a relationship between 2 categorical variables \n\n".format(chi_square_statistic, p_value))
    elif chi_square_statistic>=critical_value and p_value>alpha:
        print("chi_square_statistic is {} and p-value is {}.\nSo, there is a weak relationship between 2 categorical variables, but not evident enough \n\n".format(chi_square_statistic, p_value))
    else:
        print("chi_square_statistic is {} and p-value is {}.\nSo, these 2 variables are not related at all.\n\n".format(chi_square_statistic, p_value))

In [11]:
for i in (df.columns[:-1]):
    x=chi_sq_test(df, i, 'Safety')
    print(x, '\n')


contingency table between Buying and Safety is:

Safety  Very_Good  acceptable  good  unacceptable
Buying                                           
high           13         105     0           314
low            26          92    46           268
med            26         115    23           268
vhigh           0          72     0           360

Degree of Freedom:- 9
chi_square_statistic is 39.37083333333334 and p-value is 9.87526903328817e-06.
So, there is a relationship between 2 categorical variables 


None 

contingency table between Maintenance and Safety is:

Safety       Very_Good  acceptable  good  unacceptable
Maintenance                                           
high                20         102    18           292
low                 10          81    15           326
med                 15          99    18           300
very_high           20         102    18           292

Degree of Freedom:- 9
chi_square_statistic is 7.418269230769231 and p-value is 0.5936572365270